In [1]:
import pandas as pd
import psycopg2 as psy
import configparser as config
import pandas.io.sql as pdsql
import io
import numpy as np

class Config:
    def __init__(self):
        self.parser = config.ConfigParser()
        self.read()
     
    def read(self, file = 'Config/config.ini'):
        self.parser.read(file)
        
    def section(self, sect):
        return self.parser[sect]

class Postgres:
    def __init__(self):
        self.conf = Config()
        self.db_conf = self.conf.section('database')
        self.conn = psy.connect(self.configstr())
        
    def writedf(self, table: str,df: pd.DataFrame):
        cur = self.conn.cursor()
        bf = io.StringIO()
        bf.write(df.to_csv(index = None, header = None))
        bf.seek(0)
        cur.copy_from(bf, table,columns = df.columns, sep = ',')
        self.conn.commit()
    
    def configstr(self):
        str_config: str = ''
        for key in self.db_conf:
            str_config += key + '=\'' + self.db_conf[key] + '\' '
        
        return str_config
    
    def close(self):
        self.conn.close()
    
psql = Postgres()
    
#parser = config.ConfigParser()
#parser.read('Config/config.ini')

#db = parser['database']
#db_config: str = ''
#for key in db:
#    db_config += key + '=\'' + db[key] + '\' '

In [2]:
attendance = pd.read_csv('tmp/Attendance.csv')
attendance.index = np.arange(1, len(attendance) + 1)
attendance.drop(axis=1, columns=[x  for x in attendance.columns if x != '52023.1'],inplace=True)
attendance['student'] = pd.Series(np.arange(1, len(attendance) + 1), index=attendance.index)
attendance['id'] = pd.Series(np.arange(1, len(attendance) + 1), index=attendance.index)
attendance.rename(columns={'52023.1' : 'score'}, inplace= True)
attendance

,score,student,id
1,80,1,1
2,90,2,2
3,90,3,3
4,70,4,4
5,90,5,5
6,100,6,6
7,80,7,7
8,90,8,8
9,90,9,9
10,60,10,10


In [3]:
sample1 = pd.read_csv('tmp/Practical.csv')
sample1.drop(axis=1, columns=[x  for x in sample1.columns if x != 'roll' and x != 'total' and x != 'attendance'],inplace=True)
sample1.dropna(inplace=True)
sample1

,roll,total,attendance
0,1.0,44.0,100.0
1,2.0,41.0,100.0
2,3.0,36.0,100.0
3,4.0,39.0,50.0
4,5.0,39.0,100.0
5,6.0,38.0,100.0
6,7.0,38.0,100.0
7,8.0,37.0,100.0
8,9.0,34.0,100.0
9,10.0,41.0,0.0


In [16]:
sample2 = pd.read_csv('tmp/RollNMarks.csv')
sample2.drop(axis=1, columns= [x for x in sample2.columns if x != 'roll' and x != 'exam' and x != 'tuto' and x != 'pract'], inplace=True)
sample2.dropna(inplace=True)
sample2[:-3]

,roll,exam,tuto,pract
0,1,99.0,8.0,10.0
1,2,99.0,7.0,10.0
2,3,97.0,8.0,10.0
3,4,87.0,7.0,10.0
4,5,98.0,6.0,10.0
5,6,98.0,8.0,10.0
6,7,99.0,9.0,10.0
7,8,99.0,7.0,10.0
8,9,77.0,8.0,10.0
9,10,86.0,6.0,10.0


In [24]:
ID = np.arange(1, 30)
colStudent = pd.Series(np.arange(1, 30), index=ID)


1     44.665739
2     76.735813
3     74.799295
4     66.482483
5     17.231123
6     45.056826
7     13.087554
8     64.715454
9     83.031041
10    95.706109
11    14.358191
12    89.378705
13    96.841780
14    27.898552
15     9.174797
16    61.047084
17    56.385394
18    65.597690
19    11.882663
20    30.511121
21    48.805079
22    31.206070
23    32.071287
24    34.319903
25    42.939433
26    34.426254
27    71.764791
28    56.026266
29    23.302609
dtype: float64